In [1]:
from jpnotebooks.other.wql import *
import collections
import types
import datetime
import itertools
%load_ext autoreload
%autoreload 2

In [2]:
MIDNA = 'Mîdna'
SERVER = 'Turalyon'
OWNER = "novasan"
GUILD = "Synthesis"
ZONE = "Tomb of Sargeras"
MYTHIC = 5
api_key = "192b298b50ad53497d9f2b4ce7910af2"

try:
    zraw
except NameError:
    zraw = Api(api_key).zones._raw
    
def connect(api_key=api_key):
    global api
    api = Api(api_key)
    api.zones_ = Zones(api, zraw)
    return api

def pd(o):  # printdir
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o,a,"<Error>")
            if not isinstance(v, (types.FunctionType, types.MethodType)):
                print(a, "::", v)

                
# TODO: move these to wql.py once I'm confident they works properly


def _clstrfk(events, l):
    try:
        _clstrfk2(events, l)
    except ValueError:
        for c in l:
            for e in c:
                print(e.timestamp, actor(f, e))
            print()
    return l
    
def _clstrfk2(events, l):
    # debug sanity check
    seen = set()
    evd = dict.fromkeys(events, False)
    for c in l:
        for e in c:
            if evd[e] == True:
                raise ValueError("Duplicate event!")
            evd[e] = True
            
def cluster2(events, before=100, after=100):
    l = set()
    for e in events:
        l.add(tuple(events_near(events, e, before, after)))
    _clstrfk(events, l)
    return sorted(map(list,l), key=lambda c: c[0].timestamp)

def cluster(events, before=100, after=100):
    l = cluster2(events, before, after)
    l = [(c[0].timestamp, c) for c in l]
    return sorted(l, key=lambda t: t[0])
   
def hms2(t):
    t = t / 1000
    h = t // 3600
    m = (t-3600*h) // 60
    s = (t-3600*h-60*m)
    ms = (s - int(s))*1000
    return "%.2d:%.2d.%.3d" %(m,s, ms)
                
connect()
api.verbose = False

Requesting '/zones?api_key=192b298b50ad53497d9f2b4ce7910af2'


In [3]:
def deaths_near_bad(f, *bad, before=100, after=10000, cutoff=8):
    fs_deaths = 'type="death"'
    deaths = [e for e in f.events(fs_deaths, cutoff=cutoff) if e.targetID and f.actor(e.targetID).is_player()]
    fs_bads = 'type="damage" AND (%s)' % " OR ".join('ability.name="%s"'%b for b in bad)
    bads = f.events(fs_bads, cutoff=cutoff)
    fails = set()
    for d in deaths:
        t = tuple(events_near(bads, d, before, after)), d
        fails.add(t)
    return fails

def deaths_near_bad2(f, *bad, before=100, after=10000, cutoff=8):
    fails = deaths_near_bad(f, *bad, before=before, after=after, cutoff=cutoff)
    fails2 = set()
    deaths = set()
    for ev, d in fails:
        fails2.update(ev)
        deaths.add(d)
    key = lambda e: e.timestamp
    return sorted(fails2, key=key), sorted(deaths, key=key)

def hit_by_bad(f, *bad, cutoff=8):
    fs = 'type="damage" AND (%s)' % " OR ".join('ability.name="%s"'%b for b in bad)
    return f.events(fs, cutoff=cutoff)

In [4]:
cache = {}
def get(url):
    global cache
    if url in cache:
        return cache[url]
    rv = api.sess.request("GET", url).content.decode()
    cache[url] = rv
    return rv
def query(s):
    return pyquery.PyQuery(s)("#reports-table > tbody > tr")

In [31]:
rankings = []
p = 1
while True:
    r = api.request("/rankings/encounter/%d"%ava.id, difficulty=5, page=p, metric='speed')
    if not r['rankings']:
        break
    rankings.extend(r['rankings'])
    p += 1

In [9]:
import pyquery
s = get("https://www.warcraftlogs.com/reports/all/13?boss=2038&difficulty=5&page=1000")
q=pyquery.PyQuery(s)
q("#reports-table > tbody > tr")
q = pyquery.PyQuery(s)

In [10]:
rows = q("#reports-table > tbody > tr")
ids = []
for row in rows:
    href = row[0][0].attrib['href']
    id_ = href.split("/")[-1]
    ids.append(id_)
ids

[]

In [11]:
def get_id(row):
    href = row[0][0].attrib['href']
    return href.split("/")[-1]

In [12]:
url = "https://www.warcraftlogs.com/reports/all/13?boss=2038&difficulty=5&page=%d"
i = 1
ids = []
while True:
    print("Downloading page %d"%i)
    s = get(url%i)
    rows = query(s)
    if not rows:
        break
    for row in rows:
        id_ = get_id(row)
        ids.append(id_)
    i += 1
ids

['qXtPfhnxmg1rCzpN',
 'QCbzngVkqYA1WKhr',
 'C2gq3W1FBQ4pZwLk',
 'mk37AGqBPbtFvXf2',
 'XwmxC94bkMJqP86T',
 'XF3An8fbhYwZHLg7',
 'Gbvk93TcaBzZRxVA',
 'PVyRXqf16tmaCc48',
 'jN4WyP3MXAJfn1YD',
 'MD9PwtWT8yQafLRV',
 '2ANtdnDZvH7gWz4Q',
 'P3Cd7gBWjDRntaLk',
 'Yvgjn1pABGr36FPZ',
 'dLBDKFnvmXbQ9RJW',
 '2BQad7Xy4kZKxCfL',
 'thYQwdyWqcGPBMZD',
 'HAdnxr6LvcbP9K3V',
 'jfbKHB3TD8PM2k6Z',
 '6Xw9yLtzvQJKMZ8h',
 'ptgVzmaXdFJKw6hM',
 'gN6r93VkxWZFB8GP',
 '7YdJKAvqgPFyNjmD',
 'L3krtKnawxmQ68zb',
 'KbF3gZ8A7zjCydkL',
 'Rgyzwr8txBVNJk1m',
 'qBPA7VdRZ68xJvYc',
 'dD3Wzh9QBbXjnw8m',
 'qQa6dVDyKMX3AgJP',
 'wcXT4zHv9aR3Y8MV',
 'z4Vh3p7LTfWAxtQd',
 'K2YpkDrJGPL873Xv',
 'CB4xfYvqb3wgyMZL',
 '3cA7yNnz2drZWvpm',
 'V2qLb3AmhvGRaCzM',
 '6KdrLtZgmQ1VJfzx',
 'kG3b7mcNR8tP4CDJ',
 '1L9MvVjHaPKWwDNG',
 'jWLamJchrAYf71B3',
 '4qfPjcGH8mMKFVZh',
 'Nx9KgvkbDTVdyhBH',
 'TjYGtAa31RnykvMx',
 '4rmFxkBpnDARz2fK',
 'G1JkCNM8nLQyBKc3',
 '3QCAH69gXdNJynha',
 '2ZQ6ya1Fw4VbzkBj',
 'dV42MtF8W1zxPfHg',
 '7RW38zCBcXaHpnhq',
 'aXAYmBxW31G

In [23]:
ids_seen = set()
kills = []

In [24]:
import time
api.verbose = False
for i, id_ in enumerate(ids,1):
    if id_ in ids_seen:
        continue
    ids_seen.add(id_)
    print("\rGetting id #%d "%i, end="")
    while True:
        try:
            fights = api.fights(id_)            
            if fights.find(kill=True, difficulty=5, boss=2038).fights:
                kills.append(id_)
            else:
                # added an interim caching fix to allow 
                # manual deletion of recent cache entries
                # this call deletes the last entry added to the 
                # cache. 
                api.uncache()
        except requests.exceptions.HTTPError as e:
            if e.response.status_code == 429:  # too much spam
                print("\nToo much spam, sleeping 21 and retrying...")
                time.sleep(21)
        else:
            break

Getting id #2010 

In [128]:
count = collections.defaultdict(lambda: 0)
count = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
api.verbose=True
kf = []
specs = {}
for id_ in kills:
    for k in api.fights(id_).find(difficulty=5, boss=2038, kill=True):
        kf.append(k)
        ev = k.events('type="combatantinfo"')
        for e in ev:
            spid = e['specID']
            if spid not in specs:
                specs[spid] = e
            pid = e['sourceID']
            fr = k.friendlies.id2f[pid]
            cls = fr.type
            count[cls][SPEC_IDS[spid]] += 1
count.pop('Pet', None)

Requesting '/report/fights/aV9T2bnK3wXDdvGF?api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/aV9T2bnK3wXDdvGF?filter=type%3D%22combatantinfo%22&end=2917371&start=2444833&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/fights/VtdAXJRnG4NLcy83?api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/VtdAXJRnG4NLcy83?filter=type%3D%22combatantinfo%22&end=12745310&start=12269234&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/fights/Zvq1nCx27tTkHJcY?api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/Zvq1nCx27tTkHJcY?filter=type%3D%22combatantinfo%22&end=7704459&start=7215204&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/fights/6WZk4KxCTydLvPH9?api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/6WZk4KxCTydLvPH9?filter=type%3D%22combatantinfo%22&end=16825215&start=16348728&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/fights/DvzkpHnaR1C8rfct?api_key=192b298b50ad5349

Requesting '/report/fights/zbCpy7R1PVA6rakn?api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/zbCpy7R1PVA6rakn?filter=type%3D%22combatantinfo%22&end=11348503&start=10872137&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/fights/2TrLwZfFNkHRdaVy?api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/2TrLwZfFNkHRdaVy?filter=type%3D%22combatantinfo%22&end=19386516&start=18904610&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/fights/RM2G6TxaHZJftrLD?api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/RM2G6TxaHZJftrLD?filter=type%3D%22combatantinfo%22&end=19704567&start=19222656&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/fights/CHxGdp9yVfZgFTR1?api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/CHxGdp9yVfZgFTR1?filter=type%3D%22combatantinfo%22&end=4847720&start=4372929&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/fights/NZTC76BFhb8jvAPz?api_key=192b298b50ad53

Requesting '/report/fights/1PxWrzRZB3hMma8t?api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/1PxWrzRZB3hMma8t?filter=type%3D%22combatantinfo%22&end=10269255&start=9804108&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/fights/8wQJN2hHbzaqAgxT?api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/8wQJN2hHbzaqAgxT?filter=type%3D%22combatantinfo%22&end=6957627&start=6484733&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/fights/ZX4vkN2w1zFQVhGf?api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/ZX4vkN2w1zFQVhGf?filter=type%3D%22combatantinfo%22&end=480468&start=0&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/fights/9FhvTMa4gYkmXA1J?api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/events/9FhvTMa4gYkmXA1J?filter=type%3D%22combatantinfo%22&end=480468&start=0&api_key=192b298b50ad53497d9f2b4ce7910af2'
Requesting '/report/fights/Qk2nvpwhLM3KTABa?api_key=192b298b50ad53497d9f2b4ce7910af2'

In [141]:
%matplotlib
from matplotlib import pyplot as plt

# spec popularity
def show_spec_pop_text(count):
    l = []
    for cls in count:
        for spec in count[cls]:
            l.append((cls, spec, count[cls][spec]))
    l = sorted(l, key=lambda p: p[2], reverse=True)
    for s in l:
        print(" ".join("%-15s"%c for c in s))

# class popularity
def show_class_pop_text(count):
    l2 = []
    for cls in count:
        t = 0
        for spec in count[cls]:
            t += count[cls][spec]
        l2.append((cls, t))
    l2 = sorted(l2, key=lambda p: p[1], reverse=True)
    for s in l2:
        print(" ".join("%-15s"%c for c in s))
        
def show_spec_pie(count):
    labels = []
    values = []
    for cls in count:
        for spec in count[cls]:
            labels.append("%s %s" % (spec, cls))
            values.append(count[cls][spec])
    fig1, ax1 = plt.subplots()
    ax1.pie(values, labels=labels, autopct='%1.1f%%', shadow=False, startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    return fig

def show_class_pie(count):
    labels = []
    values = []
    for cls in count:
        c = 0
        for spec in count[cls]:
            c += count[cls][spec]
        labels.append(cls)
        values.append(c)
    fig1, ax1 = plt.subplots()
    ax1.pie(values, labels=labels, autopct='%1.1f%%', shadow=False, startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    return fig1

Using matplotlib backend: TkAgg


In [142]:
show_spec_pop_text(count)
print()
show_class_pop_text(count)
show_class_pie(count)


Mage            Fire            426            
Rogue           Assassination   365            
Rogue           Subtlety        295            
Hunter          Beast Mastery   288            
Priest          Shadow          268            
Warrior         Arms            260            
Warlock         Affliction      251            
Druid           Restoration     228            
Paladin         Holy            208            
Shaman          Restoration     192            
Druid           Balance         152            
Druid           Guardian        149            
Monk            Brewmaster      128            
Hunter          Marksmanship    103            
DeathKnight     Unholy          90             
DemonHunter     Havoc           80             
Priest          Discipline      78             
Mage            Frost           73             
Priest          Holy            65             
Warlock         Destruction     65             
Warrior         Protection      62      

NameError: name 'fig' is not defined

In [165]:
immunities = {
    62, 63, 64,
    70,
    253, 254, 255,
    259, 260, 261,
    577
}

im_count = collections.defaultdict((lambda: 0))
api.verbose=True
kkkk = None
for id_ in kills:
    for k in api.fights(id_).find(difficulty=5, boss=2038, kill=True):
        ev = k.events('type="combatantinfo"')
        nimmune = 0
        if not ev:
            print('no events!')
            kkkk = k
            continue
        for e in ev:
            spid = e['specID']
#            pid = e['sourceID']
#            fr = k.friendlies.id2f[pid]
#            cls = fr.type
            if spid in immunities:
                nimmune += 1
        if nimmune == 0:
            break
        im_count[nimmune] += 1

no events!
no events!
no events!
no events!


In [169]:
x = sorted(im_count)
y = [im_count[a] for a in x]
#x = list(map(str, x))

def autolabel(rects):
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom')

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
rects = ax.bar(x,y)
ax.set_xlabel("# of Immunity Classes")
ax.set_ylabel("# of Kills")

autolabel(rects)

In [ ]:
def throttle(t):
    def _throttle(f):
        l = time.time()
        n = l + t
        def func(*args, **kw):
            c = time.time()
            s = n-c
            if s > 0:
                time.sleep(s)
            try:
                rv = f(*args, **kw)
            finally:
                l = c
                n = c + t
            return rv
        return func
    return _throttle

@throttle(1/3)
def hf(id_):
    fights = api.fights(id_)
    if fights.find(kill=True, difficulty=5, boss=2038).fights:
        return True

In [28]:
reports = []
bosses = {}
start = datetime.datetime.now() - datetime.timedelta(weeks=1)
for r in api.reports(GUILD, SERVER, "US").find(owner=OWNER, zone=ZONE): # , func=lambda r: r.start > start):
    fights = r.fights()
    for f in fights.find(difficulty=MYTHIC):
        bosses[f.name] = bosses.get(f.name, [])
        bosses[f.name].append(f)